In [7]:
import nltk
import matplotlib
import pandas

from pdfminer.high_level import extract_text


In [9]:
file = '/Users/paul/Zotero/storage/2RVE6U8M/Shpitser et al. - 2021 - The Proximal ID Algorithm.pdf'


text = extract_text(file)
print(text)